In [120]:
import gurobipy as gp
from gurobipy import GRB
import json
import math
from prettytable import PrettyTable
from typing import List, Mapping, Tuple, Union

In [121]:
def formar_asignaciones(profesores_por_curso: Mapping[str, List[Union[List[str], str]]]) -> List[Tuple[str, str, str]]:
    """Esta función forma la lista de tuplas (curso,profesor,ramo) que deben ser asignadas de forma que puedan ser ingresadas al modelo.

    Args:
        profesores_por_curso (Mapping[str, List[List[str], str]]): Diccionario con cursos como llave. Cada curso contiene una lista que tiene a su vez una lista con el o los profesores que harán ese curso y además el nombre del curso.

    Returns:
        List[Tuple[str, str, str]]: Lista de tuplas de la forma (curso, profesor, ramo) las cuales deben ser asignadas en el horario de cada curso.
    """
    lista_asignaciones = []
    for curso in profesores_por_curso:
        for tupla in profesores_por_curso[curso]:
            for prof in tupla[0]:
                lista_asignaciones.append((curso, prof, tupla[1]))
    return lista_asignaciones

def formar_asignaciones_con_restriccion_modulos_diarios(ramos_por_curso: Mapping[str, Mapping[str, Mapping[str, int]]], asignaciones: List[Tuple[str, str, str]]) -> List[Tuple[str, str, str]]:
    """Esta función forma la lista de las clases que se quiere que necesariamente tengan módulos de clases seguidos.

    Args:
        ramos_con_modulos_seguidos (Mapping[str, Mapping[str, Mapping[str, int]]]): Diccionario con cursos como llave. Cada curso tiene asociado una lista con los ramos que se quiere que tengan módulos dobles. Cada ramo es llave de un diccionario que tiene las llaves "modulos_semanales", "maximo_modulos_diarios" y "tipo_modulos".
        asignaciones (Tuple[str, str, str]): Lista de tuplas de la forma (curso, profesor, ramo) las cuales pueden o deben ser asignadas en el horario de cada curso.

    Returns:
        Tuple[str, str, str]: Lista de tuplas de la forma (curso, profesor, ramo) que deben tener módulos seguidos de clases.
    """ 
    lista_modulos_seguidos = []
    for curso in ramos_por_curso:
        for ramo in ramos_por_curso[curso]:
            if ramos_por_curso[curso][ramo]['tipo_modulos'] == 'seguidos':
                lista_modulos_seguidos.append((curso, ramo))
    asignaciones_con_restriccion_de_modulos_diarios = []
    for elem in asignaciones:
        if (elem[0],elem[2]) in lista_modulos_seguidos:
            asignaciones_con_restriccion_de_modulos_diarios.append(elem)
    return asignaciones_con_restriccion_de_modulos_diarios

def cantidad_maxima_de_dias(carga_profesores: Mapping[str, List[Union[List[str], str]]], ramos_por_curso: Mapping[str, Mapping[str, Mapping[str, int]]]) -> Tuple[Mapping[str, int], List[str]]:
    """Esta función asigna un tope de días de trabajo a los profesores que hacen pocos módulos de clases. En la práctica esto debería ser ingresado por el usuario.

    Args:
        carga_profesores (Mapping[str, List[List[str], str]]): Diccionario con los nombres de los profesores como llave. Cada profesor tiene asociada una lista que contiene una lista de uno o más cursos a los que les hace una cierta clase conjunta. 
        ramos_por_curso (Mapping[str, Mapping[str, Mapping[str, int]]]): Diccionario con los cursos como llave. Cada uno de estos lleva a un diccionario con los ramos como llave, los cuales entregan la cantidad de módulos semanales de cada ramo y el máximo de módulos diarios permitidos.

    Returns:
        Tuple[Mapping[str, int], List[str]]: Tupla que contiene, por una parte, un diccionario con nombres de profesores como llave y entregan el máximo de días que puede trabajar cada uno y, por otra parte, una lista con todos los profesores que no tienen disponibilidad completa.
    """    
    tope_dias = {}
    for profesor in carga_profesores:
        suma = 0
        for tupla in carga_profesores[profesor]:
            suma += ramos_por_curso[tupla[0][0]][tupla[1]]['modulos_semanales']
        if suma <= 2:
            tope_dias[profesor] = 1
        elif suma <= 4:
            tope_dias[profesor] = 2
        elif suma <= 6:
            tope_dias[profesor] = 3
        elif suma <= 8:
            tope_dias[profesor] = 4
    profesores_con_dias_limitados = []
    for elem in tope_dias.keys():
        profesores_con_dias_limitados.append(elem)
    profesores_con_dias_limitados
    
    return tope_dias, profesores_con_dias_limitados

def asignaciones_no_hechas(profesores_a_definir_por_curso: Mapping[str, List[Union[List[str], str]]]) -> List[Tuple[str, str, str]]:
    """Función que forma las tuplas de los cursos que no tienen profesor definido

    Args:
        profesores_a_definir_por_curso (Mapping[str, List[List[str], str]]): Diccionario con cursos como llave. Cada curso contiene una lista que tiene a su vez una lista con el o los profesores que pueden hacer ese curso y además el nombre del curso.

    Returns:
        List[Tuple[str, str, str]]: Lista de tuplas de la forma (curso, profesor, ramo) las cuales pueden ser asignadas en el horario de cada curso.
    """    
    no_asignaciones = []
    for curso in profesores_a_definir_por_curso:
        for tupla in profesores_a_definir_por_curso[curso]:
            for prof in tupla[0]:
                no_asignaciones.append((curso, prof, tupla[1]))
    return no_asignaciones

In [122]:
model = gp.Model("Generación de horarios de colegio")
# model.setParam('OutputFlag', 1)
f = open('data/probando_formato.json',encoding='utf8')

M = 10000
data = json.load(f)
dias = data['dias']
modulos = data['modulos']
horarios = data['horarios']
profesores_por_curso = data['profesores_por_curso']
ramos_por_curso = data['ramos_por_curso']
carga_profesores = data['carga_profesores']
modulos_consecutivos = data['modulos_consecutivos']
limitaciones_profesores = data['limitaciones_profesores']
profesores_a_definir_por_curso = data['profesores_a_definir_por_curso']
asignaciones_fijadas = data['asignaciones_fijadas']
asignaciones_vetadas = data['asignaciones_vetadas']
ramos_en_distintos_dias = data['ramos_en_distintos_dias']
primeros_modulos = []
segundos_modulos = []
for j,k in modulos_consecutivos:
    primeros_modulos.append(j)
    segundos_modulos.append(k)
primeros_modulos_exclusivos = list(set(primeros_modulos) - set(segundos_modulos))
segundos_modulos_exclusivos = list(set(segundos_modulos) - set(primeros_modulos))
primeros_modulos_exclusivos, segundos_modulos_exclusivos

asignaciones_normales = formar_asignaciones(profesores_por_curso)
no_asignaciones = []
# no_asignaciones = asignaciones_no_hechas(profesores_por_curso)
total_asignaciones = asignaciones_normales + no_asignaciones
asignaciones_con_restriccion_de_modulos_diarios = formar_asignaciones_con_restriccion_modulos_diarios(
    ramos_por_curso, total_asignaciones
)
# asignaciones_multiples, cantidad_de_cursos_simultaneos = formar_asignaciones_multiples(multiples_cursos)
tope_dias, profesores_con_dias_limitados = cantidad_maxima_de_dias(carga_profesores, ramos_por_curso)


x = model.addVars(total_asignaciones,dias,modulos,vtype=GRB.BINARY, name="x")
z = model.addVars(total_asignaciones,dias,modulos,vtype=GRB.BINARY, name="z")
y = model.addVars(total_asignaciones,dias,modulos,vtype=GRB.BINARY, name="y")
# * Solo para los cursos donde se prefieren módulos dobles
w = model.addVars(no_asignaciones, vtype=GRB.BINARY, name="w")
s = model.addVars(profesores_con_dias_limitados,dias,vtype=GRB.BINARY,name="s") # * Solo para los profesores que tienen menos de 5 días
n = model.addVars(list(carga_profesores.keys()),dias,[i for i in modulos if i+4 in modulos],vtype=GRB.BINARY,name="b")

In [123]:
model.addConstrs((M * n[p,d,j] >= sum(x[c,p,r,d,j] + y[c,p,r,d,j] + z[c,p,r,d,j] for cs,r in carga_profesores[p] for c in cs) for p in carga_profesores for d in dias for j in [i for i in modulos if i+4 in modulos]),name="R0");

In [124]:
model.addConstrs((sum(x[c,ps[0],r,d,j] + z[c,ps[0],r,d,j] + y[c,ps[0],r,d,j]for ps,r in profesores_por_curso[c]) + sum(x[c,p,r,d,j] + z[c,p,r,d,j] + y[c,p,r,d,j] for ps,r in profesores_a_definir_por_curso[c] for p in ps) <= 1 for c in horarios for d in horarios[c] for j in horarios[c][d]),name="R1");

In [125]:
model.addConstrs((sum(x[c,p,r,d,j] + z[c,p,r,d,j] + y[c,p,r,d,j] for ps,r in profesores_por_curso[c] for p in ps) + sum(x[c,p,r,d,j] + z[c,p,r,d,j] + y[c,p,r,d,j] for ps,r in profesores_a_definir_por_curso[c] for p in ps) >= 1 for c in horarios for d in horarios[c] for j in horarios[c][d]),name="R2");

In [126]:
for c in profesores_por_curso:
    for ps,r in profesores_por_curso[c]:
        if len(ps) > 1:
            model.addConstrs((x[c,p1,r,d,j] - x[c,p2,r,d,j] == 0 for p1,p2 in [(ps[a],ps[b]) for a in range(len(ps)) for b in range(len(ps)) if a < b] for d in dias for j in modulos), name=f"R3[{c},{r}]");

In [127]:
for c in profesores_por_curso:
    for ps,r in profesores_por_curso[c]:
        if len(ps) > 1:
            model.addConstrs((z[c,p1,r,d,j] - z[c,p2,r,d,j] == 0 for p1,p2 in [(ps[a],ps[b]) for a in range(len(ps)) for b in range(len(ps)) if a < b] for d in dias for j in modulos), name=f"R4[{c},{r}]");

In [128]:
for c in profesores_por_curso:
    for ps,r in profesores_por_curso[c]:
        if len(ps) > 1:
            model.addConstrs((y[c,p1,r,d,j] - y[c,p2,r,d,j] == 0 for p1,p2 in [(ps[a],ps[b]) for a in range(len(ps)) for b in range(len(ps)) if a < b] for d in dias for j in modulos), name=f"R4b[{c},{r}]");

In [129]:
for c in profesores_por_curso:
    for ps,r in profesores_por_curso[c]:
        if ramos_por_curso[c][r]['tipo_modulos'] == 'seguidos':
            model.addConstrs((sum(x[c,p,r,d,j] for d in dias for j in modulos) == (ramos_por_curso[c][r]['modulos_semanales']) % 2 for p in ps if ramos_por_curso[c][r]),name=f"R5[{c},{r}]");
            model.addConstrs((sum(z[c,p,r,d,j] for d in dias for j in modulos) == math.floor((ramos_por_curso[c][r]['modulos_semanales']) / 2) for p in ps if ramos_por_curso[c][r]),name=f"R5b[{c},{r}]");
            model.addConstrs((sum(y[c,p,r,d,j] for d in dias for j in modulos) == math.floor((ramos_por_curso[c][r]['modulos_semanales']) / 2) for p in ps if ramos_por_curso[c][r]),name=f"R5c[{c},{r}]");
        else:
            model.addConstrs((sum(x[c,p,r,d,j] for d in dias for j in modulos) == ramos_por_curso[c][r]['modulos_semanales'] for p in ps if ramos_por_curso[c][r]),name=f"R5[{c},{r}]");

In [130]:
for c in profesores_a_definir_por_curso:
    for ps,r in profesores_a_definir_por_curso[c]:
        aux = []
        for p in ps:
            aux.append(p)
        aux = tuple(aux)
        model.addConstrs((sum(x[c,q,r,d,j] for q in aux for d in dias for j in modulos) == ramos_por_curso[c][r]['modulos_semanales']),name=f"R6[{c},{r}]")


In [131]:
model.addConstrs((sum(x[c,p,r,d,j] + z[c,p,r,d,j] + y[c,p,r,d,j] for ps,r in profesores_por_curso[c] for p in ps) + sum(x[c,p,r,d,j] + z[c,p,r,d,j] + y[c,p,r,d,j] for ps,r in profesores_a_definir_por_curso[c] for p in ps) == 0 for c in horarios for d in horarios[c] for j in list(set(modulos) - set(horarios[c][d]))),name="R7");

In [132]:
model.addConstrs((sum(x[cs[0],p,r,d,j] + z[cs[0],p,r,d,j] + y[cs[0],p,r,d,j] for cs,r in carga_profesores[p]) <= 1 for p in carga_profesores for d in dias for j in modulos),name="R8");

In [133]:
for p in carga_profesores:
    for cs,r in carga_profesores[p]:
        model.addConstrs((x[c1,p,r,d,j] - x[c2,p,r,d,j] == 0 for c1,c2 in [(cs[a],cs[b]) for a in range(len(cs)) for b in range(len(cs)) if a < b] for d in dias for j in modulos), name=f"R9[{p},{r}]")

In [134]:
for p in carga_profesores:
    for cs,r in carga_profesores[p]:
        model.addConstrs((z[c1,p,r,d,j] - z[c2,p,r,d,j] == 0 for c1,c2 in [(cs[a],cs[b]) for a in range(len(cs)) for b in range(len(cs)) if a < b] for d in dias for j in modulos), name=f"R10[{p},{r}]")

In [135]:
for p in carga_profesores:
    for cs,r in carga_profesores[p]:
        model.addConstrs((y[c1,p,r,d,j] - y[c2,p,r,d,j] == 0 for c1,c2 in [(cs[a],cs[b]) for a in range(len(cs)) for b in range(len(cs)) if a < b] for d in dias for j in modulos), name=f"R11[{p},{r}]")

In [136]:
for c in profesores_por_curso:
    for ps,r in profesores_por_curso[c]:
        model.addConstrs((sum(x[c,ps[0],r,d,j] + z[c,ps[0],r,d,j] + y[c,ps[0],r,d,j] for j in modulos) <= ramos_por_curso[c][r]['maximo_modulos_diarios'] for d in dias),name=f"R12[{c},{p},{r}]");

In [137]:
for c in profesores_a_definir_por_curso:
    for ps,r in profesores_a_definir_por_curso[c]:
        aux = []
        for p in ps:
            aux.append(p)
        aux = tuple(aux)
        model.addConstrs((sum(x[c,q,r,d,j] + z[c,q,r,d,j] + y[c,q,r,d,j] for q in aux for j in modulos) <= ramos_por_curso[c][r]['maximo_modulos_diarios'] for d in dias),name=f"R13[{c},{r}]")

In [138]:
model.addConstrs((x[c,p,r,d,j] + z[c,p,r,d,j] <= 1 for c,p,r in total_asignaciones if ramos_por_curso[c][r]['tipo_modulos'] == 'disjuntos' for d in dias for j,z in modulos_consecutivos), name="R14");

In [139]:
model.addConstrs((sum(x[cs[0],p,r,d,j] + z[cs[0],p,r,d,j] + y[cs[0],p,r,d,j] for cs,r in carga_profesores[p] for d in dias for j in modulos) <= limitaciones_profesores[p]['maximo_modulos'] for p in carga_profesores),name="R15");

In [140]:
model.addConstrs((sum(x[cs[0],p,r,d,j] + z[cs[0],p,r,d,j] + y[cs[0],p,r,d,j] for cs,r in carga_profesores[p] for j in modulos) <= limitaciones_profesores[p]['maximo_modulos_diario'] for p in carga_profesores for d in dias),name="R16");

In [141]:
# model.addConstrs((sum(x[cs[0],p,r,d,j1] + x[cs[0],p,r,d,j2] + x[cs[0],p,r,d,j3] + x[cs[0],p,r,d,j4] + x[cs[0],p,r,d,j5] + z[cs[0],p,r,d,j1] + z[cs[0],p,r,d,j2] + z[cs[0],p,r,d,j3] + z[cs[0],p,r,d,j4] + z[cs[0],p,r,d,j5] + y[cs[0],p,r,d,j1] + y[cs[0],p,r,d,j2] + y[cs[0],p,r,d,j3] + y[cs[0],p,r,d,j4] + y[cs[0],p,r,d,j5] for cs,r in carga_profesores[p]) <= 4 for p in carga_profesores for d in dias for j1,j2,j3,j4,j5 in [(i,i+1,i+2,i+3,i+4) for i in modulos if i+4 in modulos]), name="R17");

In [142]:
model.addConstrs((M * s[p,d] >= sum(x[cs[0],p,r,d,j] + z[cs[0],p,r,d,j] + y[cs[0],p,r,d,j] for cs,r in carga_profesores[p] for j in modulos) for d in dias for p in profesores_con_dias_limitados),name="R18");

In [143]:
model.addConstrs((sum(s[p,d] for d in dias) <= tope_dias[p] for p in profesores_con_dias_limitados),name="R19");

In [144]:
for c in profesores_a_definir_por_curso:
    for ps,r in profesores_a_definir_por_curso[c]:
        model.addConstrs((M * w[c,p,r] >= sum(x[c,p,r,d,j] + z[c,p,r,d,j] + y[c,p,r,d,j] for d in horarios[c] for j in horarios[c][d]) for p in ps),name=f"R20[{c},{r}]");

In [145]:
model.addConstrs((sum(w[c,p,r] for ps,r in profesores_a_definir_por_curso[c] for p in ps) <= 1 for c in profesores_a_definir_por_curso),name="R21");

In [146]:
model.addConstrs((x[c,p,r,d,j] + z[c,p,r,d,j] + y[c,p,r,d,j] == 1 for c,p,r,d,j in asignaciones_fijadas),name="R22");

lista_restricciones = []
for c,p,r,d,j in asignaciones_fijadas:
    lista_restricciones.append(f"R22[{c},{p},{r},{d},{j}]")

In [147]:
model.addConstrs((x[c,p,r,d,j] + z[c,p,r,d,j] + y[c,p,r,d,j] == 0 for c,p,r,d,j in asignaciones_vetadas),name="R23");

for c,p,r,d,j in asignaciones_vetadas:
    lista_restricciones.append(f"R23[{c},{p},{r},{d},{j}]")

In [148]:
for c in ramos_en_distintos_dias:
    for ls in ramos_en_distintos_dias[c]:
        for p1,r1,p2,r2 in [(ls[b][0][i],ls[b][1],ls[d][0][k],ls[d][1]) for b in range(len(ls)) for d in range(len(ls)) if b < d for i in range(len(ls[b][0])) for k in range(len(ls[d][0]))]:
            model.addConstrs((sum(x[c,p1,r1,d,j] + x[c,p2,r2,d,j] + z[c,p1,r1,d,j] + z[c,p2,r2,d,j] + y[c,p1,r1,d,j] + y[c,p2,r2,d,j] for j in modulos) <= max(ramos_por_curso[c][r1]['maximo_modulos_diarios'], ramos_por_curso[c][r2]['maximo_modulos_diarios']) for d in dias),name=f"R24[{c},{p1},{p2},{r1},{r2}]");

In [149]:
model.addConstrs((x[c,p,r,d,j] + z[c,p,r,d,j] + y[c,p,r,d,j]<= 1 for c,p,r in total_asignaciones for d in dias for j in modulos),name="R25");

In [150]:
model.addConstrs((z[c,p,r,d,j] - y[c,p,r,d,k] == 0 for c,p,r in total_asignaciones for d in dias for j,k in [(modulos[i],modulos[i+1]) for i in range(len(modulos) - 1)]),name="R28");

In [151]:
model.addConstrs((z[c,p,r,d,j] == 0 for c,p,r in total_asignaciones for d in dias for j in segundos_modulos_exclusivos),name="R29");

In [152]:
model.addConstrs((y[c,p,r,d,j] == 0 for c,p,r in total_asignaciones for d in dias for j in primeros_modulos_exclusivos),name="R30");

In [153]:
obj = sum(n[p,d,j] for p in carga_profesores for d in dias for j in [i for i in modulos if i+4 in modulos])
model.setObjective(obj, GRB.MINIMIZE)

model.setParam("PoolSolutions", 2)
model.setParam('PoolSearchMode', 2)
model.setParam('OutputFlag', 1)
model.update()

seguir = True
        
model.write('probando.lp')
# model.computeIIS()
# removed =[]
# for c in model.getConstrs():
#     if c.IISConstr:
#         print('%s' % c.constrName)
#         # Remove a single constraint from the model
#         removed.append(str(c.constrName))
#         model.remove(c)

# while seguir:
#     model.optimize()
#     if 'value' in str(model.getVars()[0]):
#         seguir = False
#     if seguir:
#         model.remove(model.getConstrByName(lista_restricciones[len(lista_restricciones) - 1]))
#         lista_restricciones.remove(lista_restricciones[len(lista_restricciones) - 1])
#         model.update()

model.update()
model.optimize()
for i in range(model.SolCount):
    model.setParam("SolutionNumber", i)
    model.update()

    print(model.optimize())
    model.write(f"solutions/out{i+1}.sol")


# model.optimize()
# model.write('out.sol')

Set parameter PoolSolutions to value 2
Set parameter PoolSearchMode to value 2
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 108434 rows, 73925 columns and 697941 nonzeros
Model fingerprint: 0x89403c37
Variable types: 0 continuous, 73925 integer (73925 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+01]
Presolve removed 104074 rows and 62467 columns
Presolve time: 0.48s
Presolved: 4360 rows, 11458 columns, 57012 nonzeros
Variable types: 0 continuous, 11458 integer (11458 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   17302    7.2932133e+02   1.720404e+05   0.000000e+00      5s
   27422    8.0650054e+02   1.623465e+03   0.000000e+00     10s
   28841    8.0650000e+02   0.000000e+00   0.000000e+00     11s

Root relaxatio

In [154]:
def leer_output(numero_sol):
    lista_final = []
    with open(f'solutions/out{numero_sol}.sol', 'r') as file:
        lines = file.readlines()[2:]
        for line in lines:
            if line[-2] == '1':
                if line[-4] == ']':
                    line = line.strip('\n')
                    lista_final.append(line)
                else:
                    line = line.strip('\n')
                    if round(float(line[line.find(']') + 2:])) == 1:
                        lista_final.append(line[:line.find(']') + 1] + ' 1')
            else:
                if line[-4] != ']':
                    line = line.strip('\n')
                    if int(float(line[line.find(']') + 2:])) == 1:
                        lista_final.append(line[:line.find(']') + 1] + ' 1')

    diccionario_dias = {
        'Lunes': 1,
        'Martes': 2,
        'Miercoles': 3,
        'Jueves': 4,
        'Viernes': 5
    }

    vuelta = {
        1: 'Lunes',
        2: 'Martes',
        3: 'Miercoles',
        4: 'Jueves',
        5: 'Viernes'
    }

    profesores_final = []
    for elem in lista_final:
        if elem[0] == "x" or elem[0] == "y" or elem[0] == "z":
            elem = elem[2:-3]
            elem = elem.split(',')
            profesores_final.append(elem)
            
    for i in range(len(profesores_final)):
        profesores_final[i][3] = diccionario_dias[profesores_final[i][3]]
    profesores_final = sorted(profesores_final, key= lambda x: int(x[4]))
    profesores_final = sorted(profesores_final, key= lambda x: x[3])
    profesores_final = sorted(profesores_final, key= lambda x: x[0])

    for i in range(len(profesores_final)):
        profesores_final[i][3] = vuelta[profesores_final[i][3]]

    ultimo_curso = None
    ultimo_ramo = None
    ultimo_dia = None
    ultimo_modulo = None
    indices = []
    for i in range(len(profesores_final)):
        if profesores_final[i][0] == ultimo_curso:
            if profesores_final[i][2] == ultimo_ramo:
                if profesores_final[i][3] == ultimo_dia:
                    if profesores_final[i][4] == ultimo_modulo:
                        indices.append(i)
        ultimo_curso = profesores_final[i][0]
        ultimo_ramo = profesores_final[i][2]
        ultimo_dia = profesores_final[i][3]
        ultimo_modulo = profesores_final[i][4]
    indices.reverse()
    for indice in indices:
        profesores_final[indice - 1][1] = profesores_final[indice - 1][1] + ' & ' + profesores_final[indice][1]
        profesores_final.pop(indice)
    return profesores_final

def generar_horario(curso, numero_sol):

    clases_curso = []
    dias_semana = []
    ultimo_dia = None
    horario = PrettyTable(dias_semana)
    lista_auxiliar = []
    lista_dia = None
    for elem in leer_output(numero_sol):    
        if elem[0] == curso:
            clases_curso.append([elem[2], elem[3], elem[4], elem[1]])
            if elem[3] not in dias_semana:
                dias_semana.append(elem[3])
            if elem[3] != ultimo_dia:
                lista_auxiliar.append(lista_dia)
                lista_dia = []
                ultimo_dia = elem[3]
                lista_dia.append(ultimo_dia)
                elem[2] = elem[2].replace('_', ' ')
                elem[1] = elem[1].replace('_', ' ')
                lista_dia.append(f'{elem[2]} \n {elem[1]}')
            else:
                elem[2] = elem[2].replace('_', ' ')
                elem[1] = elem[1].replace('_', ' ')
                lista_dia.append(f'{elem[2]} \n {elem[1]}')

    lista_auxiliar.append(lista_dia)
    lista_auxiliar = lista_auxiliar[1:]
    len_maxima = 0
    for dia in lista_auxiliar:
        if len(dia) > len_maxima:
            len_maxima = len(dia)

    horario.add_column('', [i+1 for i in range(len_maxima - 1)])
    
    for i in range(len(lista_auxiliar)):
        for j in range(len_maxima - len(lista_auxiliar[i])):
            lista_auxiliar[i].append('')
        horario.add_column(lista_auxiliar[i][0], lista_auxiliar[i][1:], align='c')
    horario.format = True   
    print(f'Horario del curso {curso}: ')   
    return horario

curso = '5A'
generar_horario(curso,numero_sol=2)

Horario del curso 5A: 


,Lunes,Martes,Miercoles,Jueves,Viernes
1,Ingles1 Ivan Aguayo,Ingles1 Ivan Aguayo,Tutoria Maria Paz Videla,Ingles1 Ivan Aguayo,Artes Visuales/Tecnologia Sara Sanchez
2,Ingles1 Ivan Aguayo,Orientacion Sara Sanchez,Tutoria Maria Paz Videla,Ingles1 Ivan Aguayo,Artes Visuales/Tecnologia Sara Sanchez
3,Educacion Fisica y Salud Erik X & Joaquina X,Historia Geografia y CS Patricio Santibañez,Ciencias Naturales Marcela Bravo,Lenguaje y Comunicacion Nicole Jofre,Historia Geografia y CS Patricio Santibañez
4,Educacion Fisica y Salud Erik X & Joaquina X,Historia Geografia y CS Patricio Santibañez,Ciencias Naturales Marcela Bravo,Lenguaje y Comunicacion Nicole Jofre,Historia Geografia y CS Patricio Santibañez
5,Lenguaje y Comunicacion Nicole Jofre,Ciencias Naturales Marcela Bravo,Lenguaje y Comunicacion Nicole Jofre,Ingles2 Claudia Astudillo,Ingles1 Ivan Aguayo
6,Lenguaje y Comunicacion Nicole Jofre,Ciencias Naturales Marcela Bravo,Lenguaje y Comunicacion Nicole Jofre,Ingles2 Claudia Astudillo,Ingles1 Ivan Aguayo
7,Crecer y Crecer/Religion Maria Jose X & Cristian X,Musica Patricio Vidal & Ramiro X,Matematica Erika Albornoz & Jessica Diaz,Educacion Fisica y Salud Erik X & Joaquina X,
8,Crecer y Crecer/Religion Maria Jose X & Cristian X,Musica Patricio Vidal & Ramiro X,Matematica Erika Albornoz & Jessica Diaz,Educacion Fisica y Salud Erik X & Joaquina X,
9,,Matematica Erika Albornoz & Jessica Diaz,,Matematica Erika Albornoz & Jessica Diaz,
10,,Matematica Erika Albornoz & Jessica Diaz,,Matematica Erika Albornoz & Jessica Diaz,


: 

In [12]:
def generar_horario_profesor(nombre_profesor,numero_sol):
    clases_profesor = []
    dict_dias = {'Lunes': 1, 'Martes': 2, 'Miercoles': 3, 'Jueves': 4, 'Viernes': 5, 'Sábado': 6, 'Domingo': 7}
    dict_dias_inverso = {1: 'Lunes', 2: 'Martes', 3: 'Miercoles', 4: 'Jueves', 5: 'Viernes', 6: 'Sábado', 7: 'Domingo'}
    for elem in leer_output(numero_sol):
        if elem[1] == nombre_profesor:
            clases_profesor.append(elem)
        elif '&' in elem[1]:
            if nombre_profesor in elem[1]:
                clases_profesor.append([elem[0], nombre_profesor, elem[2], elem[3], elem[4]])
            
    for i in range(len(clases_profesor)):
        clases_profesor[i][3] = dict_dias[clases_profesor[i][3]]
    clases_profesor = sorted(clases_profesor, key= lambda x: int(x[4]))
    clases_profesor = sorted(clases_profesor, key= lambda x: x[3])
    for i in range(len(clases_profesor)):
        clases_profesor[i][3] = dict_dias_inverso[clases_profesor[i][3]]

    ingreso_clase = False
    lista_auxiliar = []
    for i in dias:
        lista_dia = [i]
        for j in modulos:
            if clases_profesor == []:
                lista_dia.append(' ')
            elif clases_profesor != []:
                ingreso_clase = False
                primera = True
                if i != clases_profesor[0][3] or j != int(clases_profesor[0][4]):
                    lista_dia.append(' ')          
                while i == clases_profesor[0][3] and j == int(clases_profesor[0][4]):
                    ingreso_clase = True
                    if primera:
                        cursos = f'{clases_profesor[0][0]}'
                    else:
                        cursos += f' & {clases_profesor[0][0]}'
                    ramo = clases_profesor[0][2]
                    clases_profesor.pop(0)
                    primera = False
                    if clases_profesor == []:
                        break
                if ingreso_clase:
                    lista_dia.append(f'{cursos} \n {ramo}')
        lista_auxiliar.append(lista_dia)

    # print(dias)
    horario = PrettyTable([])
    horario.add_column('', modulos)
    # print(modulos)
    # print(horario)
    for i in range(len(dias)):
        # print(lista_auxiliar)
        horario.add_column(lista_auxiliar[i][0], lista_auxiliar[i][1:], align='c')
    horario.format = True   
    print(f'Horario del profesor {nombre_profesor}: ')   
    return horario

generar_horario_profesor(nombre_profesor='Eraldo_Cortes', numero_sol=1)

Horario del profesor Eraldo_Cortes: 


,Lunes,Martes,Miercoles,Jueves,Viernes
1,IIIA Matematica,IIIB Matematica,IIB Matematica1,IVB Matematica_M1-1,IIIB Orientacion
2,IIIA Matematica,IIIB Matematica,IIB Matematica1,IVB Matematica_M1-1,IB Matematica_PDT
3,IIB Matematica1,,IIIA Matematica,IA Matematica1,IVA Matematica_M1-1
4,IIB Matematica1,IIB Matematica1,IIIA Matematica,,IVA Matematica_M1-1
5,IVA Matematica_M1-1,,IVB Matematica_M1-1,IIIB Matematica,IIB Matematica1
6,IVA Matematica_M1-1,,IVB Matematica_M1-1,IIIB Matematica,IIB Matematica1
7,IIIA & IIIB Geom3D/BMolecular/Vida/Composicion,,IA Matematica1,IIIA & IIIB Geom3D/BMolecular/Vida/Composicion,
8,IIIA & IIIB Geom3D/BMolecular/Vida/Composicion,IIA Matematica2,IA Matematica1,IIIA & IIIB Geom3D/BMolecular/Vida/Composicion,
9,IA Matematica1,IA Matematica1,IIIA & IIIB Geom3D/BMolecular/Vida/Composicion,IIB Matematica2,
10,IA Matematica1,IA Matematica1,IIIA & IIIB Geom3D/BMolecular/Vida/Composicion,,
